<a href="https://colab.research.google.com/github/MatteoZanella/NLU-assignement-2/blob/main/NLU_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLU assignment n.2

Update SpaCy to version 3 and download the dataset

In [1]:
%%capture
!pip install --upgrade spacy
!python -m spacy download en_core_web_sm
!wget -nc https://raw.githubusercontent.com/esrel/NLU.Lab.2021/master/src/conll.py
!wget -nc https://github.com/esrel/NLU.Lab.2021/raw/master/src/conll2003.zip
!unzip -n conll2003.zip -d conll2003

Load the dataset and Spacy

In [2]:
# Imports
import random
import conll
import spacy
import pandas as pd
from spacy.tokens import Token
from spacy.training import Alignment
from sklearn.metrics import classification_report


nlp = spacy.load("en_core_web_sm")

gt = conll.read_corpus_conll("./conll2003/train.txt", fs=" ")
gt.extend(conll.read_corpus_conll("./conll2003/test.txt", fs=" "))
gt.extend(conll.read_corpus_conll("./conll2003/dev.txt", fs=" "))

# Removing reference lines
gt = [tag_sent for tag_sent in gt if tag_sent[0][0] != '-DOCSTART-']

# Limit the dataset, for a faster analysis in the following code
gt = random.sample(gt, 8000)

## Task 1: SpaCy NER evaluation
Evaluate spaCy NER on CoNLL 2003 data (provided)


### Part 1.1: SpaCy NER and alignement
As first step, we need to evaluate the sentences with SpaCy

Creation of custom extentions to save the dataset information directly in the SpaCy tokens

In [3]:
Token.set_extension("ent_ref", default='')

Translation function: By scanning the entire Conll dataset, you can see that the only Entities present are:
`'LOC', 'ORG', 'PER', 'MISC'`

SpaCy Entities, more detailed, should be translated according to their meaning

In [4]:
def to_ref_entity(token):
  ent_iob = token.ent_iob_
  ent_type = token.ent_type_
  if ent_type == 'ORG' : # Organizations
    ent_type = 'ORG'
  elif ent_type == 'PERSON':  # Persons
    ent_type = 'PER'
  elif ent_type == 'GPE' or ent_type == 'FAC' or ent_type == 'LOC':  # Localities
    ent_type = 'LOC'
  else:
    ent_type = 'MISC'
  
  if ent_iob == 'O':
    return ent_iob 
  else:
    return f"{ent_iob}-{ent_type}"

The spacy tokenization is different from the one provided in the dataset.
I checked `alignment.x2y.lengths` and verified that spacy tokens needs to be merged at most, never to be splitted.

In [5]:
docs = []
for gt_sentence in gt:
  # List of ground truth tokens (token, POS, chunk, entity)
  gt_tokens = [tup[0] for tup in gt_sentence]
  # Create Doc object and extract tokens
  doc = nlp(" ".join(gt_tokens))
  doc_tokens = [t.text for t in doc]
  
  # Get the alignment: .y2x.lengths has the merge informations
  # .x2y.lengths is all ones with the tokenization considered
  alignment = Alignment.from_strings(doc_tokens, gt_tokens)
  # Merge together tokens to reflect ground truth tokenization
  with doc.retokenize() as retokenizer:
    doc_idx = 0
    for length in alignment.y2x.lengths:
      if length > 1:
        retokenizer.merge(doc[doc_idx:doc_idx+length])
      doc_idx += length

  # Add the information about chunk division and entity
  for token, ref in zip(doc, gt_sentence):
    token._.ent_ref = ref[3]
  docs.append(doc)

### Part 1.2: Token-level performance
Report token-level performance (per class and total)
  - accuracy of correctly recognizing all tokens that belong to named entities (i.e. tag-level accuracy)
  - to get per-class and total token-level performances you use scikit-learn's classification report, like we did in the lab on evaluation (you don't need to compute accuracy per-class, such thing does not exist)


In [6]:
def token_entities(docs):
  """Extract token-level predicted and reference Named Entities, as requested by classification_report"""
  token_NE_ref = []
  token_NE_pred = []

  for doc in docs:
    for token in doc:
      token_NE_ref.append(token._.ent_ref)
      token_NE_pred.append(to_ref_entity(token))
  return token_NE_ref, token_NE_pred

In [7]:
# Print the results
NE_ref, NE_pred = token_entities(docs)
print(classification_report(NE_ref, NE_pred))
print('='*80)
# Optional Confusion Matrix
y_actu = pd.Series(NE_ref, name='Actual')
y_pred = pd.Series(NE_pred, name='Predicted')
pd_tbl = pd.crosstab(y_pred, y_actu)
pd_tbl.round(decimals=3)

              precision    recall  f1-score   support

       B-LOC       0.79      0.69      0.74      4025
      B-MISC       0.12      0.57      0.20      1993
       B-ORG       0.48      0.31      0.37      3588
       B-PER       0.80      0.66      0.72      3920
       I-LOC       0.55      0.59      0.57       651
      I-MISC       0.06      0.31      0.09       722
       I-ORG       0.45      0.54      0.49      1971
       I-PER       0.82      0.82      0.82      2670
           O       0.95      0.87      0.91     97195

    accuracy                           0.82    116735
   macro avg       0.56      0.60      0.55    116735
weighted avg       0.89      0.82      0.85    116735



Actual,B-LOC,B-MISC,B-ORG,B-PER,I-LOC,I-MISC,I-ORG,I-PER,O
Predicted,,,,,,,,,
B-LOC,2787,50,406,120,7,8,17,6,113
B-MISC,84,1142,72,53,0,33,22,2,7880
B-ORG,187,194,1102,326,2,8,22,4,457
B-PER,94,47,357,2588,3,6,15,40,82
I-LOC,78,18,12,5,383,21,94,30,56
I-MISC,3,66,21,16,19,221,23,21,3552
I-ORG,64,66,254,40,92,156,1071,131,501
I-PER,9,3,15,43,20,23,217,2184,155
O,719,407,1349,729,125,246,490,252,84399


### Part 1.3: Chunk-level performance
Report CoNLL chunk-level performance (per class and total):
  - Precision, recall, f-measure of correctly recognizing all the named entities in a chunk per class and total.
  - To get chunk-level NER performance, you simply need to use conll.py's evaluate, that computes segmentation and labeling performance.



In [8]:
def chunk_entities(docs):
  """Transform tokens into (text, iob), as requested by conll.evaluate()"""
  chunk_NE_ref = []
  chunk_NE_pred = []
  
  for doc in docs:
    chunk_NE_pred.append([(t.text, to_ref_entity(t)) for t in doc])
    chunk_NE_ref.append([(t.text, t._.ent_ref) for t in doc])
  
  return chunk_NE_ref, chunk_NE_pred

In [9]:
# Print the results
NE_ref, NE_pred = chunk_entities(docs)
results = conll.evaluate(NE_ref, NE_pred)
pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
LOC,0.781,0.681,0.728,4025
MISC,0.116,0.540,0.191,1993
ORG,0.429,0.275,0.335,3588
PER,0.778,0.642,0.703,3920
total,0.399,0.541,0.460,13526


## Task 2: Grouping of Entities
Write a function to group recognized named entities using noun_chunks method of [spaCy](https://spacy.io/usage/linguistic-features#noun-chunks).

In [10]:
def grouped_entities(sentence):
  doc = nlp(sentence)
  n_chunks = list(doc.noun_chunks)
  entities = []
  
  curr_chunk = 0
  curr_token = 0
  chunk_ents = set()
  for token in doc:
    if curr_chunk < len(n_chunks) and token == n_chunks[curr_chunk][curr_token]:  # Token is next in the noun chunk
      if token.ent_type_ != '': # Middle or final token
        chunk_ents.add(token.ent_type_)
      curr_token += 1
      if token == n_chunks[curr_chunk][-1]:  # Last token of the current chunk
        if len(chunk_ents) > 0:
          entities.append(sorted(chunk_ents))  # Sorted, so it's a list without notion of token ordering
        curr_chunk += 1  # Look for next token
        curr_token = 0  # At the first position
        chunk_ents = set()  # Reset the set of chunk's tokens
    elif token.ent_type_ != '':
      entities.append([token.ent_type_])
  return entities

In [11]:
# Testing the function
grouped_entities("Apple's Steve Jobs died in 2011 in Palo Alto, California.")

[['ORG', 'PERSON'], ['DATE'], ['GPE'], ['GPE']]

### Part 2.1: Frequency analysis
Analyze the groups in terms of most frequent combinations (i.e. NER types that go together).

From the ground truth dataset, extract the corpus with all plain text sentences

In [12]:
corpus = [" ".join([tup[0] for tup in gt_sentence]) for gt_sentence in gt]

In [13]:
from collections import Counter

frequencies = Counter()
for sentence in corpus: 
  entities = grouped_entities(sentence)
  for group in entities:
    combination = '-'.join(group)
    frequencies[combination] += 1

In [14]:
for combination, counter in frequencies.most_common():
  print(f"{combination}: {counter}")

DATE: 4865
CARDINAL: 3901
GPE: 3079
PERSON: 3052
ORG: 2417
NORP: 864
MONEY: 547
ORDINAL: 422
TIME: 411
QUANTITY: 222
PERCENT: 209
LOC: 116
EVENT: 107
NORP-PERSON: 106
FAC: 94
WORK_OF_ART: 86
CARDINAL-PERSON: 79
GPE-PERSON: 71
PRODUCT: 67
CARDINAL-ORG: 67
ORG-PERSON: 64
GPE-ORG: 49
CARDINAL-NORP: 38
LAW: 38
DATE-ORG: 32
DATE-PERSON: 29
DATE-TIME: 25
NORP-ORG: 24
DATE-GPE: 23
CARDINAL-GPE: 23
NORP-ORDINAL: 20
LANGUAGE: 18
DATE-NORP: 17
GPE-NORP: 17
DATE-EVENT: 15
ORDINAL-PERSON: 15
GPE-ORDINAL: 15
CARDINAL-DATE: 12
ORDINAL-ORG: 10
ORG-PRODUCT: 8
GPE-PRODUCT: 8
CARDINAL-ORDINAL: 7
LANGUAGE-ORDINAL: 6
MONEY-ORG: 5
ORDINAL-QUANTITY: 5
DATE-ORDINAL: 4
EVENT-PERSON: 4
EVENT-NORP: 4
CARDINAL-PRODUCT: 4
FAC-GPE: 4
PERSON-TIME: 3
DATE-WORK_OF_ART: 3
DATE-NORP-PERSON: 3
CARDINAL-EVENT: 3
EVENT-ORDINAL: 3
DATE-MONEY: 3
CARDINAL-PERCENT: 3
DATE-PERCENT: 3
LOC-NORP: 2
PERSON-WORK_OF_ART: 2
CARDINAL-GPE-PERSON: 2
GPE-MONEY: 2
LOC-PERSON: 2
GPE-ORG-PERSON: 2
DATE-QUANTITY: 2
GPE-PERCENT: 2
GPE-TIME: 2

## Task 3: Covering full noun-compounds
One of the possible post-processing steps is to fix segmentation errors. Write a function that extends the entity span to cover the full noun-compounds. Make use of compound dependency relation.

### Part 3.1: Definition and application

You have to be careful when extending entities with the coumpound, because you could overwrite other entities.

In [15]:
from spacy.tokens import Span

def expand_entities(doc):
  entities = []
  for ents_i, ent in enumerate(doc.ents):
    ent_start = ent.start
    ent_end = ent.end
    # List of all the children of the entity span tokens
    subtree = list(ent.root.subtree) 
    search_start = subtree[0].i
    search_end = subtree[-1].i + 1
    # The search should be limited by previous and next entities
    if ents_i > 0 and doc.ents[ents_i - 1].end > search_start:
      search_start = doc.ents[ents_i - 1].end
    if ents_i < (len(doc.ents) - 1) and doc.ents[ents_i + 1].start < search_end:
      search_end = doc.ents[ents_i + 1].start
    # Extend the head
    token = doc[search_start]
    while token.i < ent_start:
      compound_root = token
      while compound_root.dep_ == 'compound' and not (ent_start <= compound_root.i < ent_end):
        compound_root = compound_root.head
      if ent_start <= compound_root.i < ent_end:
        ent_start = token.i
      token = token.nbor()
    # Extend the tail
    token = doc[search_end - 1]
    while token.i >= ent_end:
      compound_root = token
      while compound_root.dep_ == 'compound' and not (ent_start <= compound_root.i < ent_end):
        compound_root = compound_root.head
      if ent_start <= compound_root.i < ent_end:
        ent_end = token.i + 1
      token = token.nbor(-1)
    # Add the expanded entity to the list
    entity = Span(doc, ent_start, ent_end, label=ent.label_)
    entities.append(entity)
  # Set the extended entities
  doc.set_ents(entities)

We can directy apply the postprocessing to the docs object.

In [16]:
# Application of the post-processing step
for doc in docs:
  expand_entities(doc)

### Part 3.2: Evaluation

Results are worse. For instance, "Shimon Peres" is extended to "minister Shimon Peres" since minister has a compound relationship with Shimon, and that's clearly not the correct identification of the PERSON named entity

In [17]:
# Evaluation of the results
NE_ref, NE_pred = token_entities(docs)
print(classification_report(NE_ref, NE_pred))
print('='*80)

NE_ref, NE_pred = chunk_entities(docs)
results = conll.evaluate(NE_ref, NE_pred)
pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

              precision    recall  f1-score   support

       B-LOC       0.78      0.68      0.73      4025
      B-MISC       0.12      0.57      0.20      1993
       B-ORG       0.47      0.30      0.36      3588
       B-PER       0.67      0.55      0.61      3920
       I-LOC       0.49      0.60      0.54       651
      I-MISC       0.06      0.31      0.09       722
       I-ORG       0.43      0.55      0.48      1971
       I-PER       0.66      0.83      0.73      2670
           O       0.95      0.86      0.90     97195

    accuracy                           0.81    116735
   macro avg       0.51      0.58      0.52    116735
weighted avg       0.88      0.81      0.84    116735



,p,r,f,s
LOC,0.767,0.670,0.715,4025
MISC,0.116,0.539,0.190,1993
ORG,0.416,0.267,0.325,3588
PER,0.648,0.534,0.586,3920
total,0.372,0.504,0.428,13526
